## 1. Import & load libraries


In [5]:
%pip cache purge

%pip install mne


Files removed: 0 (0 bytes)
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.python.org/simple
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip list


Package            Version
------------------ -----------
asttokens          3.0.0
certifi            2025.1.31
charset-normalizer 3.4.1
colorama           0.4.4
comm               0.2.2
contourpy          1.3.1
cycler             0.12.1
debugpy            1.8.12
decorator          5.1.1
exceptiongroup     1.2.2
executing          2.1.0
fonttools          4.56.0
idna               3.10
importlib_metadata 8.6.1
ipykernel          6.29.5
ipython            8.32.0
jedi               0.19.2
Jinja2             3.1.5
joblib             1.4.2
jupyter_client     8.6.3
jupyter_core       5.7.2
kiwisolver         1.4.8
lazy_loader        0.4
MarkupSafe         3.0.2
matplotlib         3.9.4
matplotlib-inline  0.1.7
mne                1.9.0
nest_asyncio       1.6.0
numpy              2.0.2
packaging          24.2
pandas             2.2.3
parso              0.8.4
pexpect            4.9.0
pickleshare        0.7.5
pillow             11.1.0
pip                25.0
platformdirs       4.3.6
pooch      

In [7]:
import mne
from mne.preprocessing import ICA
# from mne import Epochs, pick_types, events_from_annotations
# from mne.channels import make_standard_montage
# from mne.io import concatenate_raws, read_raw_edf
# from mne.datasets import eegbci
# from mne.decoding import CSP

from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import cross_val_score, GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
# from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
# from sklearn.exceptions import ConvergenceWarning

import numpy as np
import pandas as pd

import warnings
import json
import os
import time

import pickle

# from tqdm import tqdm

# from IPython.display import display, HTML

from typing import List




## 2. Load data

In [8]:
# Load data from models directory
models_dir = '../models'

# Load preprocessed data and pipeline
X = np.load(os.path.join(models_dir, 'X_preprocessed.npy'))
y = np.load(os.path.join(models_dir, 'y_labels.npy'))

# Load trained pipeline
with open(os.path.join(models_dir, 'trained_pipeline.pkl'), 'rb') as f:
    pipeline = pickle.load(f)

# 1. Split data into train, validation and test sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print("Dataset shapes:")
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

# 2. Cross-validation on training data
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)
print("\nCross-validation results:")
print(f"Scores: {cv_scores}")
print(f"Mean CV score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

# 3. Train final model
pipeline.fit(X_train, y_train)

# 4. Evaluate on validation set
val_score = pipeline.score(X_val, y_val)
print(f"\nValidation set accuracy: {val_score:.3f}")

# 5. Real-time prediction simulation
def evaluate_real_time(pipeline, X, y, delay_threshold=2.0):
    predictions = []
    times = []
    
    for i in range(len(X)):
        start_time = time.time()
        
        # Make prediction
        pred = pipeline.predict(X[i].reshape(1, -1))
        
        # Record prediction time
        pred_time = time.time() - start_time
        times.append(pred_time)
        predictions.append(pred[0])
        
        # Check if prediction is within time limit
        if pred_time > delay_threshold:
            print(f"Warning: Prediction {i} took {pred_time:.3f}s (exceeds {delay_threshold}s threshold)")
    
    accuracy = np.mean(np.array(predictions) == y)
    avg_time = np.mean(times)
    
    return accuracy, avg_time, predictions, times

print("\nEvaluating real-time performance on test set...")
test_accuracy, avg_pred_time, predictions, times = evaluate_real_time(pipeline, X_test, y_test)
print(f"Test set accuracy: {test_accuracy:.3f}")
print(f"Average prediction time: {avg_pred_time:.3f}s")

# 6. Save evaluation results
evaluation_results = {
    'dataset_split': {
        'train_size': len(X_train),
        'validation_size': len(X_val),
        'test_size': len(X_test)
    },
    'cross_validation': {
        'scores': cv_scores.tolist(),
        'mean_score': float(cv_scores.mean()),
        'std_score': float(cv_scores.std())
    },
    'validation_score': float(val_score),
    'test_performance': {
        'accuracy': float(test_accuracy),
        'avg_prediction_time': float(avg_pred_time),
        'max_prediction_time': float(max(times)),
        'min_prediction_time': float(min(times))
    }
}

# Save evaluation results
with open(os.path.join(models_dir, 'evaluation_results.json'), 'w') as f:
    json.dump(evaluation_results, f, indent=4)

# Print summary
print("\nSummary:")
print(f"Training set size: {len(X_train)}")
print(f"Validation accuracy: {val_score:.3f}")
print(f"Test accuracy: {test_accuracy:.3f}")
print(f"Average prediction time: {avg_pred_time:.3f}s")
print(f"Results saved to: {os.path.join(models_dir, 'evaluation_results.json')}")

# Verify 60% accuracy requirement
if test_accuracy >= 0.6:
    print("\nSuccess: Model meets the 60% accuracy requirement!")
else:
    print("\nWarning: Model does not meet the 60% accuracy requirement.")
    print("Consider tuning hyperparameters or trying different preprocessing steps.")


Dataset shapes:
Training set: (9, 51264)
Validation set: (3, 51264)
Test set: (3, 51264)

Cross-validation results:
Scores: [0.5 0.5 0.5 0.5 1. ]
Mean CV score: 0.600 (+/- 0.400)


/sgoinfre/students/dgerwig-/miniforge/envs/vortex_env/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(



Validation set accuracy: 0.333

Evaluating real-time performance on test set...
Test set accuracy: 0.333
Average prediction time: 0.001s

Summary:
Training set size: 9
Validation accuracy: 0.333
Test accuracy: 0.333
Average prediction time: 0.001s
Results saved to: ../models/evaluation_results.json

Consider tuning hyperparameters or trying different preprocessing steps.
